## Get a filtration from Gudhi

In [6]:
import gudhi as gd
import tadasets
sphere = tadasets.dsphere(n=10, d=1, r=1, noise=0, seed=42)
alpha = gd.AlphaComplex(points=sphere)
st = alpha.create_simplex_tree()
filtration = list(st.get_filtration())

## Convert it to boundary map data

In [7]:
from persistent_laplacians.filtrations import simplices_by_dimension, compute_boundary_matrices, filtration_hash_map

unique_filtration_values = sorted(list(set([f for (_, f) in filtration])))
simplices_by_dim, simplices_by_dim_only_filt = simplices_by_dimension(filtration)
boundary_matrices = compute_boundary_matrices(simplices_by_dim)
boundary_maps_index_dict = filtration_hash_map(filtration, simplices_by_dim_only_filt)

## Compute homology using persistent laplacians
For now, compute only homology (using SVD)

In [8]:
from persistent_laplacians import persistent_laplacians as pl
result = pl.process_tda(
    boundary_matrices,
    boundary_maps_index_dict
)

## Barcodes 

In [9]:
from persistent_laplacians.barcodes import compute_barcodes
barcodes = compute_barcodes(result, unique_filtration_values)
print(barcodes)
barcodes = [
    (q, (unique_filtration_values[i], unique_filtration_values[j]))
    for q in barcodes.keys()
    for (i, j) in barcodes[q].keys()
]
sorted(barcodes)

{2: {}, 1: {(14, 15): 1, (10, 16): 1, (12, 13): 1, (14, 16): 1}}


[(1, (0.3065431994272291, 1.0)),
 (1, (0.9999999999999953, 0.9999999999999973)),
 (1, (0.9999999999999996, 0.9999999999999997)),
 (1, (0.9999999999999996, 1.0))]

## Verify with Gudhi

In [10]:
gudhi_persistence = [(q, (i, j)) for (q, (i,j)) in st.persistence() if q > 0]
sorted(gudhi_persistence) == sorted(barcodes)

True